In [1]:
import torch
from torch import optim
import torchvision
from torchvision import models
from torchvision.transforms import v2
import torch.nn as nn
from PIL import Image
import pandas as pd
print(torchvision.__version__)
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

0.18.0+cu121


In [2]:
classes = ('anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')

In [3]:
def preprocess(image):
    transform = v2.Compose([
        v2.ToImage() ,
        v2.ToDtype(torch.uint8, scale=True),
        v2.CenterCrop((96, 96)),
        v2.ToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    return image


In [4]:
def create_one_hot_encoding(label, classes):
    one_hot = torch.zeros(len(classes), dtype=torch.float32)
    
    if label in classes:
        index = classes.index(label)
        one_hot[index] = 1.0
    return one_hot

In [5]:
      
class CustomFERDataset(Dataset):
    def __init__(self, image_parent_directory, data_directory, transform=None):
        self.image_parent_directory = image_parent_directory
        self.df = pd.read_csv(data_directory)
        self.transform = transform
        self.classes = classes

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.image_parent_directory + self.df.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.df.iloc[idx, 1]
        label_tensor = torch.tensor(create_one_hot_encoding(label, self.classes))
        
        return image, label_tensor

In [6]:
train_data = CustomFERDataset("data/archive/" , "data/train.csv" , transform=preprocess)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

test_data = CustomFERDataset("data/archive/" , "data/test.csv" , transform=preprocess)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [7]:
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [13]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data, target
            output = model(data)
            
            test_loss += criterion(output, target).item()  # sum up batch loss
            
            # Convert one-hot encoded target to class labels if needed
            if target.ndim > 1:
                _, target = target.max(1)  # convert one-hot encoded target to class labels
                
            # Calculate correct predictions
            _, pred = output.max(1)  # get the index of the max log-probability
            correct += pred.eq(target).sum().item()
            
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')


In [9]:
class SimpleResNet(nn.Module):
    def __init__(self, num_classes=8):
        super(SimpleResNet, self).__init__()
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)  # Modify the last layer for CIFAR-10

    def forward(self, x):
        return self.model(x)

In [10]:
model = SimpleResNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion)

# Save the model
torch.save(model.state_dict(), "simple_resnet.pth")

  0%|          | 0/705 [00:00<?, ?it/s]/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/tmp/ipykernel_5004/1003653074.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = 

Train Epoch: 1 [0/22540 (0%)]	Loss: 2.013803


 14%|█▍        | 101/705 [01:05<07:11,  1.40it/s]

Train Epoch: 1 [3200/22540 (14%)]	Loss: 1.946120


 29%|██▊       | 201/705 [01:56<03:57,  2.12it/s]

...

 85%|████████▌ | 601/705 [04:05<00:42,  2.45it/s]

Train Epoch: 10 [19200/22540 (85%)]	Loss: 0.134896


 99%|█████████▉| 701/705 [04:46<00:01,  2.51it/s]

Train Epoch: 10 [22400/22540 (99%)]	Loss: 0.048313


100%|██████████| 705/705 [04:48<00:00,  2.45it/s]
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/tmp/ipykernel_5004/1003653074.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = torch.tensor(create_one_hot_encoding(label, self.classes))
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=

In [14]:
test_model = SimpleResNet()
test_model.load_state_dict(torch.load("simple_resnet.pth"))
criterion = nn.CrossEntropyLoss()
test(test_model, test_loader, criterion)


/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/tmp/ipykernel_5004/1003653074.py:19: UserWarning: To copy construct from a tensor, 


Test set: Average loss: 0.0513, Accuracy: 1744/2818 (61.89%)

